**Packages** **used**

In [1]:
import pandas
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
...

Ellipsis

# Uploading File

In [2]:
from google.colab import files
uploaded= files.upload()

Saving WQI_Parameter_Scores_1994-2013.csv to WQI_Parameter_Scores_1994-2013.csv


In [3]:
# load dataset
dataframe = pandas.read_csv("WQI_Parameter_Scores_1994-2013.csv")
dataset = dataframe.values
dataframe.head()

,Station,Station Name,Year,Overall WQI,WQI FC,WQI Oxy,WQI pH,WQI TSS,WQI Temp,WQI TPN,WQI TP,WQI Turb,Location 1
0,16A070,Skokomish R nr Potlatch,1996.0,87,91,85,95,78,81,99,84,69,POINT (-123.1771 47.3098)
1,16A070,Skokomish R nr Potlatch,1997.0,86,78,84,77,85,90,99,72,80,POINT (-123.1771 47.3098)
2,16A070,Skokomish R nr Potlatch,1998.0,75,90,79,88,61,90,98,71,49,POINT (-123.1771 47.3098)
3,16A070,Skokomish R nr Potlatch,1999.0,87,88,80,88,75,95,100,79,71,POINT (-123.1771 47.3098)
4,16A070,Skokomish R nr Potlatch,2000.0,95,93,86,96,84,90,100,84,87,POINT (-123.1771 47.3098)


**This is a multiple class problem and can be solved using Artificial Neural Networks.**
**Based on designations set by CCME(2005) to classify water based on WQI.**
```
Excellent /1    95-100     All measurements are within objectives virtually all of the time
```
```
Good /2          80-94      Conditions rarely depart from natural or desirable levels
```
```
Fair /3          65-79      Conditions sometimes depart from natural or desirable levels
```
```
Marginal /4     45-64      Conditions often depart from natural or desirable levels
```
```
Poor /5          0-44      Conditions usually depart from natural or desirable levels**
```

In [4]:
def quality_check(num):
    s=0
    if num <=100 and num >=80 :
        s=1
    #elif num <=89 and num >=80:
        #s= 2
    elif num <=79 and num >=65:
        s=2
    elif num <=64 : #and num >=50
        s=3
    #elif num <=49:
     #   s=5
    return s

In [5]:
dataframe['quality']=dataframe['Overall WQI'].apply(quality_check)

In [6]:
dataframe

,Station,Station Name,Year,Overall WQI,WQI FC,WQI Oxy,WQI pH,WQI TSS,WQI Temp,WQI TPN,WQI TP,WQI Turb,Location 1,quality
0,16A070,Skokomish R nr Potlatch,1996.0,87,91,85,95,78,81,99,84,69,POINT (-123.1771 47.3098),1
1,16A070,Skokomish R nr Potlatch,1997.0,86,78,84,77,85,90,99,72,80,POINT (-123.1771 47.3098),1
2,16A070,Skokomish R nr Potlatch,1998.0,75,90,79,88,61,90,98,71,49,POINT (-123.1771 47.3098),2
3,16A070,Skokomish R nr Potlatch,1999.0,87,88,80,88,75,95,100,79,71,POINT (-123.1771 47.3098),1
4,16A070,Skokomish R nr Potlatch,2000.0,95,93,86,96,84,90,100,84,87,POINT (-123.1771 47.3098),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,23A070,Chehalis R @ Porter,2013.0,62,85,89,92,68,68,52,58,75,POINT (-123.3138 46.9393),3
967,23A160,Chehalis R @ Dryad,2013.0,65,76,78,96,78,65,77,73,81,POINT (-123.2501 46.6309),2
968,24B090,Willapa R nr Willapa,2013.0,80,78,87,91,78,78,57,69,82,POINT (-123.6535 46.6501),1
969,24F070,Naselle R nr Naselle,2013.0,79,67,88,94,87,77,67,88,93,POINT (-123.7468 46.3729),2


In [8]:
print(dataframe.loc[dataframe['quality']==4])

Empty DataFrame
Columns: [Station, Station Name, Year, Overall WQI, WQI FC, WQI Oxy, WQI pH, WQI TSS, WQI Temp, WQI TPN, WQI TP, WQI Turb, Location 1, quality]
Index: []


In [9]:
dataset = dataframe.values

In [10]:
X = dataset[:,4:11].astype(float)
Y = dataset[:,13]


In [12]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

## 1 - hot encoding *since multi class problem

In [13]:
dummy_y

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train, x_test, y_train, y_test = train_test_split( X, dummy_y, test_size=0.2)

In [16]:
dummy_y.shape

(971, 3)

## **Creating an artificial neural network with two hidden layers**
Softmax function has been used to calculate the probability in the output layer.
THe two hidden layers consists 16 hidden units each.The input layer consists of 7 features that are available from the data set -- as discussed earlier.

In [17]:
def baseline_model():
  classifier2= Sequential()
  classifier2.add(Dense(16,input_dim=7, activation='relu'))
  classifier2.add(Dense(16, activation='relu'))
  classifier2.add(Dense(5, activation='softmax'))
  classifier2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return classifier2  

In [18]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=1000, batch_size=10, verbose=0)

## K- fold Cross - Validation

In [19]:
kfold = KFold(n_splits=10, shuffle=True)

In [31]:
#results = cross_val_score(estimator, X, dummy_y, cv=kfold)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [27]:
classifier1= Sequential()
classifier1.add(Dense(14,input_dim=7, activation='relu'))
classifier1.add(Dense(10, activation='relu'))
classifier1.add(Dense(10, activation='relu'))
classifier1.add(Dense(3, activation='softmax'))
classifier1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier1.fit(x_train,y_train, epochs=400, batch_size=4, verbose=1)


Epoch 1/400
194/194 [==============================] - 0s 3ms/step - loss: 3.0808 - accuracy: 0.5052
Epoch 2/400
194/194 [==============================] - 0s 3ms/step - loss: 0.9736 - accuracy: 0.5606
Epoch 3/400
194/194 [==============================] - 1s 3ms/step - loss: 0.7964 - accuracy: 0.6534
Epoch 4/400
194/194 [==============================] - 1s 3ms/step - loss: 0.7239 - accuracy: 0.6856
Epoch 5/400
194/194 [==============================] - 1s 3ms/step - loss: 0.6771 - accuracy: 0.6959
Epoch 6/400
194/194 [==============================] - 0s 3ms/step - loss: 0.6536 - accuracy: 0.7049
Epoch 7/400
194/194 [==============================] - 1s 3ms/step - loss: 0.6114 - accuracy: 0.7281
Epoch 8/400
194/194 [==============================] - 0s 3ms/step - loss: 0.6119 - accuracy: 0.7255
Epoch 9/400
194/194 [==============================] - 0s 3ms/step - loss: 0.5949 - accuracy: 0.7564
Epoch 10/400
194/194 [==============================] - 0s 2ms/step - loss: 0.5870 - accura

In [28]:
score = classifier1.evaluate(x_test, y_test, verbose = 1) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

7/7 [==============================] - 0s 3ms/step - loss: 0.5209 - accuracy: 0.7846
Test loss: 0.5208988785743713
Test accuracy: 0.7846153974533081


**Results : Validation accuracy is 72.8% and test accuracy is 78.4%**